## Knowledge Graph
#### Drug-Food or Drug-Supplements interaction prediction

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from pykeen.models import predict
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import TransE
from pykeen.evaluation import RankBasedEvaluator

from os import listdir

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Split data into train, validation and test sets

In [2]:
data_dir = 'data/triplets/'

In [3]:
ddi_df = pd.read_csv(data_dir + 'ddi.tsv', sep='\t', index_col=[0])
ddi_df.head()

,drug1,interaction,drug2
0,Apixaban,increase_anticoagulant_activities,Lepirudin
1,Dabigatran etexilate,increase_anticoagulant_activities,Lepirudin
2,Dasatinib,increase_bleeding,Lepirudin
3,Dasatinib,increase_hemorrhage,Lepirudin
4,Deferasirox,increase_gastrointestinal_bleeding,Lepirudin


In [4]:
print('total interactions:', ddi_df.shape[0])
print('unique interactions:', len(set(ddi_df.interaction)))

ddi_df.groupby(by=['interaction']).size()

total interactions: 3123450
unique interactions: 205


interaction
decrease_absorption                             576
decrease_adverse_effects                        384
decrease_anticholinergic_activities              26
decrease_anticoagulant_activities              5174
decrease_antihypertensive_activities          63420
                                              ...  
increase_visual_accommodation_disturbances        2
increase_vomiting                                54
increase_water_intoxication                     256
increase_weakness                              1710
increase_weight_gain                             16
Length: 205, dtype: int64

In [5]:
ddi_df.groupby(by=['interaction']).size().min()

2

In [6]:
drug_supplement_df = pd.read_csv(data_dir + 'ds_relations.tsv', sep='\t', index_col=[0])
drug_supplement_df = drug_supplement_df[drug_supplement_df['REL'] != 'has_ingredient']
drug_supplement_df.head()

,CUI1,REL,CUI2
689297,DC0477348,has_adverse_effect_on,DC0478433
689298,DC0477348,has_adverse_effect_on,DC0478434
689299,DC0477348,has_adverse_effect_on,DC0478432
689300,DC0477348,has_adverse_effect_on,DC0478435
689301,DC0477348,interacts_with,DC0478436


In [7]:
print('total interactions:', drug_supplement_df.shape[0])
print('unique interactions:', len(set(drug_supplement_df.REL)))

drug_supplement_df.groupby(by=['REL']).size()

total interactions: 18867
unique interactions: 5


REL
has_adverse_effect_on    3120
has_adverse_reaction     2093
has_therapeutic_class    5443
interacts_with           3057
is_effective_for         5154
dtype: int64

In [8]:
def compute_size(n):
    if n == 2:
        return 1, 1
    if n == 3:
        return 1, 2
    if n == 4:
        return 2, 3
    if n == 5:
        return 3, 4
    if n == 6:
        return 4, 5
    # n == 7
    return 4, 6 

In [9]:
# train : valid : test = 80 : 10 : 10
def split_data_relation(df_relation):
    if df_relation.shape[0] <= 7:
        train_size, valid_size = compute_size(df_relation.shape[0])
        
        df_relation = df_relation.sample(frac=1, random_state=42)
        X_train = df_relation.iloc[:train_size]
        X_valid = df_relation.iloc[train_size:valid_size]
        X_test = df_relation.iloc[valid_size:]

    else:
        X_train, X_rem = train_test_split(df_relation, train_size=0.8, random_state=42)
        X_valid, X_test = train_test_split(X_rem, test_size=0.5, random_state=42)
        
    return X_train, X_valid, X_test

In [10]:
# split drug-supplements relation dataset
def split_drug_supplements_dataset(drug_supplement_df):
    relations = set(drug_supplement_df.REL)
    
    train_triplets = pd.DataFrame(columns=['CUI1', 'REL', 'CUI2'])
    valid_triplets = pd.DataFrame(columns=['CUI1', 'REL', 'CUI2'])
    test_triplets = pd.DataFrame(columns=['CUI1', 'REL', 'CUI2'])

    for rel in relations:
        train, valid, test = split_data_relation(drug_supplement_df[drug_supplement_df['REL'] == rel])
        train_triplets = pd.concat([train_triplets, train])
        valid_triplets = pd.concat([valid_triplets, valid])
        test_triplets = pd.concat([test_triplets, test])

    train_triplets.rename(columns={'CUI1': 'head', 'REL': 'relation', 'CUI2': 'tail'}, inplace=True)
    valid_triplets.rename(columns={'CUI1': 'head', 'REL': 'relation', 'CUI2': 'tail'}, inplace=True)
    test_triplets.rename(columns={'CUI1': 'head', 'REL': 'relation', 'CUI2': 'tail'}, inplace=True)    


    print('train dataset size:', train_triplets.shape[0])
    print('validation dataset size:',valid_triplets.shape[0])
    print('test dataset size:',test_triplets.shape[0])
    
    return train_triplets, valid_triplets, test_triplets

In [11]:
# split drug-drug interaction dataset (from DrugBank)
def split_ddi_dataset(ddi_df):
    interactions = set(ddi_df.interaction)
    
    train_triplets = pd.DataFrame(columns=['drug1', 'interaction', 'drug2'])
    valid_triplets = pd.DataFrame(columns=['drug1', 'interaction', 'drug2'])
    test_triplets = pd.DataFrame(columns=['drug1', 'interaction', 'drug2'])
    
    for inter in interactions:
        train, valid, test = split_data_relation(ddi_df[ddi_df['interaction'] == inter])
        train_triplets = pd.concat([train_triplets, train])
        valid_triplets = pd.concat([valid_triplets, valid])
        test_triplets = pd.concat([test_triplets, test])
        
    train_triplets.rename(columns={'drug1': 'head', 'interaction': 'relation', 'drug2': 'tail'}, inplace=True)
    valid_triplets.rename(columns={'drug1': 'head', 'interaction': 'relation', 'drug2': 'tail'}, inplace=True)
    test_triplets.rename(columns={'drug1': 'head', 'interaction': 'relation', 'drug2': 'tail'}, inplace=True)    


    print('train dataset size:', train_triplets.shape[0])
    print('validation dataset size:',valid_triplets.shape[0])
    print('test dataset size:',test_triplets.shape[0])
    
    return train_triplets, valid_triplets, test_triplets    
    

In [12]:
# DrugBank drug-drug interactions
print('DrugBank drug-drug interactions')
train_triplets_ddi, valid_triplets_ddi, test_triplets_ddi = split_ddi_dataset(ddi_df)

# Drug Supplement database - drug-suplement interactions
print('Drug Supplement database - drug-suplement interactions')
train_triplets_ds, valid_triplets_ds, test_triplets_ds = split_drug_supplements_dataset(drug_supplement_df)


DrugBank drug-drug interactions
train dataset size: 2498661
validation dataset size: 312349
test dataset size: 312440
Drug Supplement database - drug-suplement interactions
train dataset size: 15092
validation dataset size: 1886
test dataset size: 1889


In [13]:
# all interactions
train_triplets = pd.concat([train_triplets_ddi, train_triplets_ds])
valid_triplets = pd.concat([valid_triplets_ddi, valid_triplets_ds])
test_triplets = pd.concat([test_triplets_ddi, test_triplets_ds])

print('All interactions:')
print('train dataset size:', train_triplets.shape[0])
print('validation dataset size:',valid_triplets.shape[0])
print('test dataset size:',test_triplets.shape[0])

All interactions:
train dataset size: 2513753
validation dataset size: 314235
test dataset size: 314329


#### Add rest of the data into the training set

In [14]:
files = listdir(data_dir)

for file in files:
    if file == 'ddi.tsv':
        continue
        
    if file == 'ds_relations.tsv':
        df = pd.read_csv(data_dir + file, sep='\t', index_col=[0])
        df = df[df['REL'] == 'has_ingredient']
        
    else:    
        df = pd.read_csv(data_dir + file, sep='\t', index_col=[0])
    
    df.set_axis(['head', 'relation', 'tail'], axis=1, inplace=True) # copy=False insead of inplace -> newer versionn of pandas?
    train_triplets = pd.concat([train_triplets, df])
    
print('Final size of train dataset (with other relations):', train_triplets.shape[0]) 

/tmp/ipykernel_1899/1327732482.py:14: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['head', 'relation', 'tail'], axis=1, inplace=True) # copy=False insead of inplace -> newer versionn of pandas?
/tmp/ipykernel_1899/1327732482.py:14: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['head', 'relation', 'tail'], axis=1, inplace=True) # copy=False insead of inplace -> newer versionn of pandas?
/tmp/ipykernel_1899/1327732482.py:14: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['head', 'relation', 'tail'], axis=1, inplace=True) # copy=False insead of inplace -> newer versionn of pandas?
/tmp/ipykernel_1899/1327732482.py:14:

Final size of train dataset (with other relations): 20657032


In [15]:
train_triplets = train_triplets.astype(str)
valid_triplets = valid_triplets.astype(str)
test_triplets = test_triplets.astype(str)

In [16]:
# save train, valid and test datasets

# train_triplets.to_csv(data_dir + 'train.tsv', sep='\t')
# valid_triplets.to_csv(data_dir + 'valid.tsv', sep='\t')
# test_triplets.to_csv(data_dir + 'test.tsv', sep='\t')

#### Convert datasets into Triples Factory format

In [17]:
def convert_to_triples_factory(data):
    tf_data = TriplesFactory.from_labeled_triples(
      data[["head", "relation", "tail"]].values,
      create_inverse_triples=False,
      entity_to_id=None,
      relation_to_id=None,
      compact_id=False 
    )
    print(tf_data)  # kam mizeji nejake trojice? - jiny pocet zde a po vytvoreni datasetu
    return tf_data

In [18]:
tf_train = convert_to_triples_factory(train_triplets)
tf_valid = convert_to_triples_factory(valid_triplets)
tf_test = convert_to_triples_factory(test_triplets)


TriplesFactory(num_entities=667535, num_relations=221, create_inverse_triples=False, num_triples=4646665)
TriplesFactory(num_entities=5629, num_relations=187, create_inverse_triples=False, num_triples=314106)
TriplesFactory(num_entities=5593, num_relations=210, create_inverse_triples=False, num_triples=314223)


In [21]:
# take just subset of data for testing

train_sub, _ = tf_train.split(0.15)
valid_sub, _ = tf_valid.split(0.15)
test_sub, _ = tf_test.split(0.15)

using automatically assigned random_state=3719750636
using automatically assigned random_state=3467520645
using automatically assigned random_state=2864710704


In [22]:
train_sub

TriplesFactory(num_entities=667535, num_relations=221, create_inverse_triples=False, num_triples=696999)

### Model

In [ ]:
# epochs=1 device=gpu?

In [ ]:
# creating a model
result = pipeline(
    training=train_sub,
    testing=test_sub,
    validation=valid_sub,
    model='TransE',
    evaluator=RankBasedEvaluator
)

No random seed is specified. Setting to 501653776.
No cuda devices were available. The model runs on CPU
Training batches on cpu:   2%|█▏                                            | 67/2723 [00:19<12:45,  3.47batch/s]


Training batches on cpu:   5%|██▎                                          | 137/2723 [00:39<12:25,  3.47batch/s]


Training batches on cpu:   8%|███▍                                         | 207/2723 [00:59<12:09,  3.45batch/s]


Training batches on cpu:  10%|████▌                                        | 277/2723 [01:20<11:46,  3.46batch/s]


Training batches on cpu:  13%|█████▋                                       | 347/2723 [01:40<11:28,  3.45batch/s]


...

Training batches on cpu:  91%|███████████████████████████████████████▊    | 2467/2723 [12:17<01:16,  3.35batch/s]


Training batches on cpu:  93%|████████████████████████████████████████▉   | 2537/2723 [12:38<00:55,  3.33batch/s]


Training batches on cpu:  96%|██████████████████████████████████████████▏ | 2607/2723 [12:59<00:34,  3.37batch/s]


Training batches on cpu:  98%|███████████████████████████████████████████▎| 2677/2723 [13:19<00:13,  3.34batch/s]


Training epochs on cpu: 100%|█████████████████| 5/5 [1:06:58<00:00, 803.63s/epoch, loss=0.0184, prev_loss=0.0462]
INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.
Evaluating on cpu:  17%|███████▋                                      | 7.90k/47.1k [22:27<1:51:42, 5.85triple/s]

In [ ]:
hits_at_10 = result.get_metric('hits@10')

In [ ]:
# predictions

model = result.model

predictions_df = predict.get_tail_prediction_df(model, 'DC0477356', 'has_adverse_effect_on', triples_factory=result.training)
predictions_df

In [ ]:
# evaluation
# ????
evaluator = RankBasedEvaluator()

triples = tf_test.mapped_triples

eval_results = evaluator.evaluate(
    model=model,
    mapped_triples=triples,
    batch_size=1024,
    additional_filter_triples=[
       tf_train.mapped_triples,
       tf_valid.mapped_triples,
    ],
)

In [ ]:
eval_results.to_df()

In [ ]:
result.metric_results.to_df()

In [ ]:
result.save_to_directory("results")